# Asthma

---

## Starting up: Connecting BigQuery Data pipeline for Google Colab

The following set of codes is used to create the framework for testing and clustering analysis.

### **Why BigQuery Used for Handling Big Data in this Project**

**Scalability and Performance:** BigQuery is a fully-managed data warehouse designed for large-scale data analysis. It excels in handling massive datasets efficiently, which is essential for big data analytics.

**Serverless Architecture:** BigQuery's serverless architecture eliminates the need for infrastructure management. Users can focus on analyzing data without worrying about the underlying hardware or database maintenance.

**Speed:** Thanks to its distributed architecture, BigQuery can process queries on large datasets at an impressive speed. This makes it ideal for real-time analytics and rapid insights.

**SQL Interface:** It provides a familiar SQL interface for querying data, making it accessible to analysts and data scientists with SQL knowledge.

**Security and Compliance:** With robust security features and compliance with many international standards, BigQuery ensures that data is protected and managed according to best practices.

Cost-Effective: **bold text** Its pay-as-you-go pricing model allows for cost-effective analysis of big data, where users pay only for the data processed by their queries, without the need for upfront costs.

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'capstone-400517' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=capstone-400517:US:bquxjob_bfbd722_18b725d9f3e)
back to BigQuery to edit the query within the BigQuery user interface.

In [ ]:
## Query to get LOS AND TOTAL CHARGE FOR PATIENTS WITH ASTHMA AND OTHER DISEASES

query = """

SELECT fc.*, kg.DXCCSR_Default_DX1
FROM `capstone-400517.capstone2.for_clustering` AS fc
LEFT JOIN `capstone-400517.capstone2.kid_GPRS` AS kg
ON fc.recnum = kg.recnum

"""


In [ ]:
# Running this code will display the query used to generate your previous job

job = client.query(query) # Job ID inserted based on the query results selected to explore
print(job.query)



SELECT fc.*, kg.DXCCSR_Default_DX1
FROM `capstone-400517.capstone2.for_clustering` AS fc
LEFT JOIN `capstone-400517.capstone2.kid_GPRS` AS kg
ON fc.recnum = kg.recnum




## Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [ ]:
# Filtering only for RSP009
results = job.to_dataframe()
results = results[results['DXCCSR_Default_DX1']== 'RSP009']

## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [ ]:
results

,RECNUM,FEMALE,AGE,RACE,HOSP_REGION,ZIPINC_QRTL,LOS,TOTCHG,PAY1
711,21004198,0,1,6,2,1,0,11232,2
712,10735922,0,3,3,1,1,2,21705,2
713,10421936,0,4,3,1,1,1,14302,2
714,10188450,1,2,6,1,1,0,15688,2
715,10049572,1,3,2,1,1,4,37820,2
...,...,...,...,...,...,...,...,...,...
3087021,30324392,0,6,2,3,NaN,2,11678,2
3087068,30550598,0,10,2,3,NaN,0,11147,3
3087108,40024278,1,9,3,4,NaN,0,23896,4
3087109,30331205,0,11,2,3,NaN,1,12501,2


In [ ]:
# prompt: Using dataframe results: delete column DXCCSR_Default_DX1

results = results.drop(columns=['DXCCSR_Default_DX1'], axis=1, inplace=True)


## Data Cleaning and Preprocessing

In [ ]:
# Import numpy to handle NaN values
import numpy as np

# Replace all instances of 'nan' with NaN in the results DataFrame
results.replace('nan', np.nan, inplace=True)


In [ ]:
# Show the results
results.sample(12)

,RECNUM,FEMALE,AGE,RACE,HOSP_REGION,ZIPINC_QRTL,LOS,TOTCHG,PAY1
1079751,40969470,0,5,6,4,3,3,58009,3
1033378,30618462,0,12,NaN,3,3,2,17765,2
1007489,40126907,0,12,2,4,3,2,42399,2
243640,30155571,0,13,1,3,2,0,22935,2
2772646,10140259,0,15,3,1,2,2,12956,2
2181711,10781474,1,19,1,1,2,2,39394,2
3059019,10620283,1,1,4,1,4,3,49027,2
522130,10104208,0,4,1,1,4,1,9001,6
2568311,21051620,1,10,2,2,1,2,27732,2
2276962,10217501,0,2,2,1,3,1,7069,3


## Handling missing values

In [ ]:
# Drop NA values from table
results.dropna(inplace=True)

In [ ]:
# Show the results
results.sample(12)

,RECNUM,FEMALE,AGE,RACE,HOSP_REGION,ZIPINC_QRTL,LOS,TOTCHG,PAY1
1777910,10720999,0,0,1,1,4,1,20439,3
1422592,30988723,0,1,2,3,1,2,10623,2
722504,30585512,1,11,2,3,1,1,16900,2
730015,10241323,0,11,2,1,1,2,16047,2
2542608,30599591,0,7,1,3,1,1,4735,2
101747,31628519,0,3,1,3,1,3,18753,2
1576018,41094764,1,19,2,4,3,2,35444,2
2678663,20804466,1,5,1,2,2,3,18898,3
1719193,10450089,0,4,3,1,3,0,6730,3
1492095,10573038,0,10,2,1,2,2,17746,2


In [ ]:
results.replace('A', np.nan, inplace=True)

In [ ]:
# Show unique values in each column values
cols = results.columns
for i in cols:

  print(i, results[i].unique())

RECNUM ['21004198' '10735922' '10421936' ... '31792643' '10633122' '41248820']
FEMALE ['0' '1']
AGE ['1' '3' '4' '2' '6' '12' '9' '10' '7' '17' '19' '8' '14' '0' '15' '16'
 '20' '5' '18' '13' '11']
RACE ['6' '3' '2' '1' '4' '5']
HOSP_REGION ['2' '1' '3' '4']
ZIPINC_QRTL ['1' '2' '3' '4' nan]
LOS ['0' '2' '1' '4' '3' '5' '6' '9' '7' '8' '12' '10' '11' '29' '20' '25'
 '14' '53' '13' '16' '19' '28' '17' '15' '51' '72' '38' '18' '24' '216'
 '21' '71' '26' '62' '22' '39' '37' '99']
TOTCHG ['11232' '21705' '14302' ... '11271' '13177' '95767']
PAY1 ['2' '3' '6' '4' '1' '5' nan]


In [ ]:
# Drop NA values form table
results.dropna(inplace=True)

In [ ]:
# check if we have NA values among unique values

cols = results.columns
for i in cols:

  print(i, results[i].unique())

RECNUM ['21004198' '10735922' '10421936' ... '31792643' '10633122' '41248820']
FEMALE ['0' '1']
AGE ['1' '3' '4' '2' '6' '12' '9' '10' '7' '17' '19' '8' '14' '0' '15' '16'
 '20' '5' '18' '13' '11']
RACE ['6' '3' '2' '1' '4' '5']
HOSP_REGION ['2' '1' '3' '4']
ZIPINC_QRTL ['1' '2' '3' '4']
LOS ['0' '2' '1' '4' '3' '5' '6' '9' '7' '8' '12' '10' '11' '29' '20' '25'
 '14' '53' '13' '16' '19' '28' '17' '15' '51' '72' '38' '18' '24' '216'
 '21' '71' '26' '62' '22' '39' '37' '99']
TOTCHG ['11232' '21705' '14302' ... '11271' '13177' '95767']
PAY1 ['2' '3' '6' '4' '1' '5']


In [ ]:
#Converting all values into numeric so that we can run ML algorithms

import pandas as pd

# Convert all columns to numeric, coercing errors to NaN
results = results.apply(pd.to_numeric, errors='coerce')


# Convert to Int64
results = results.astype('Int64')

In [ ]:
results.describe()

,RECNUM,FEMALE,AGE,RACE,HOSP_REGION,ZIPINC_QRTL,LOS,TOTCHG,PAY1
count,44524.0,44524.0,44524.0,44524.0,44524.0,44524.0,44524.0,44521.0,44524.0
mean,24401504.536273,0.409824,6.696186,2.231426,2.361333,2.102888,2.08099,23419.566946,2.45791
std,10537184.499163,0.491807,5.055742,1.263211,1.033879,1.105934,2.061821,40222.253378,0.763445
min,10000017.0,0.0,0.0,1.0,1.0,1.0,0.0,106.0,1.0
25%,10832384.25,0.0,3.0,1.0,1.0,1.0,1.0,10204.0,2.0
50%,30062988.0,0.0,5.0,2.0,3.0,2.0,2.0,16477.0,2.0
75%,31594966.5,1.0,10.0,3.0,3.0,3.0,3.0,27238.0,3.0
max,41346065.0,1.0,20.0,6.0,4.0,4.0,216.0,5415370.0,6.0


In [ ]:
#Checking if the values converted into numeric

cols = results.columns
for i in cols:

  print(i, results[i].unique())

RECNUM <IntegerArray>
[21004198, 10735922, 10421936, 10188450, 10049572, 10112288, 20894492,
 10924329, 10754919, 10173260,
 ...
 31846372, 10869094, 10738005, 20965790, 20087074, 20271440, 10142096,
 31792643, 10633122, 41248820]
Length: 44524, dtype: Int64
FEMALE <IntegerArray>
[0, 1]
Length: 2, dtype: Int64
AGE <IntegerArray>
[1, 3, 4, 2, 6, 12, 9, 10, 7, 17, 19, 8, 14, 0, 15, 16, 20, 5, 18, 13, 11]
Length: 21, dtype: Int64
RACE <IntegerArray>
[6, 3, 2, 1, 4, 5]
Length: 6, dtype: Int64
HOSP_REGION <IntegerArray>
[2, 1, 3, 4]
Length: 4, dtype: Int64
ZIPINC_QRTL <IntegerArray>
[1, 2, 3, 4]
Length: 4, dtype: Int64
LOS <IntegerArray>
[  0,   2,   1,   4,   3,   5,   6,   9,   7,   8,  12,  10,  11,  29,  20,
  25,  14,  53,  13,  16,  19,  28,  17,  15,  51,  72,  38,  18,  24, 216,
  21,  71,  26,  62,  22,  39,  37,  99]
Length: 38, dtype: Int64
TOTCHG <IntegerArray>
[11232, 21705, 14302, 15688, 37820, 10630,  8896, 20560, 11983, 15150,
 ...
 84063, 33222, 41229, 14658, 92157, 36343, 

## Changing data types into numeric

In [ ]:
# Checking data types of each columns

results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44524 entries, 711 to 3080440
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   RECNUM       44524 non-null  Int64
 1   FEMALE       44524 non-null  Int64
 2   AGE          44524 non-null  Int64
 3   RACE         44524 non-null  Int64
 4   HOSP_REGION  44524 non-null  Int64
 5   ZIPINC_QRTL  44524 non-null  Int64
 6   LOS          44524 non-null  Int64
 7   TOTCHG       44521 non-null  Int64
 8   PAY1         44524 non-null  Int64
dtypes: Int64(9)
memory usage: 3.8 MB


In [ ]:
# Separating the features that  we need for K-Mode algorthm

df_kmode = results[['AGE','FEMALE', 'RACE', 'HOSP_REGION', 'ZIPINC_QRTL', 'LOS', 'TOTCHG', 'PAY1']]

In [ ]:
# Creating bins for Length of stay column

## labels = ['0-1', '2-3', '4-9', '10 and over']


bins = [-1, 1, 3, 9, float('inf')]  # Using -1 as the lower bound to include 0
labels = [1, 2, 3, 4]

# Create a new column with the binned categories
df_kmode.loc[:, 'LOS'] = pd.cut(df_kmode['LOS'], bins=bins, labels=labels)


In [ ]:
# Checking if the bins created

df_kmode

,AGE,FEMALE,RACE,HOSP_REGION,ZIPINC_QRTL,LOS,TOTCHG,PAY1
711,1,0,6,2,1,1,11232,2
712,3,0,3,1,1,2,21705,2
713,4,0,3,1,1,1,14302,2
714,2,1,6,1,1,1,15688,2
715,3,1,2,1,1,3,37820,2
...,...,...,...,...,...,...,...,...
3080170,1,1,3,2,4,1,13177,6
3080388,11,0,4,1,4,3,95767,3
3080389,14,0,1,3,4,2,17393,6
3080390,20,0,2,1,4,2,13134,2


## Creating bins for continuous values

In [ ]:
# Creating bins for "Total charge" feature

## labels = ['0-12000', '12,000-20,000' '20,000-40,000', '40,000-80,000', '80,000+']


bins = [-1, 12000, 20000, 40000, 80000, float('inf')]  # Using -1 as the lower bound to include 0
labels = [1, 2, 3, 4, 5]

# Create a new column with the binned categories
df_kmode.loc[:, 'TOTCHG'] = pd.cut(df_kmode['TOTCHG'], bins=bins, labels=labels)

<ipython-input-60-12ae8d966eb2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kmode.loc[:, 'TOTCHG'] = pd.cut(df_kmode['TOTCHG'], bins=bins, labels=labels)
<ipython-input-60-12ae8d966eb2>:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_kmode.loc[:, 'TOTCHG'] = pd.cut(df_kmode['TOTCHG'], bins=bins, labels=labels)


In [ ]:
# check if the bins created

df_kmode

,AGE,FEMALE,RACE,HOSP_REGION,ZIPINC_QRTL,LOS,TOTCHG,PAY1
711,1,0,6,2,1,1,1,2
712,3,0,3,1,1,2,3,2
713,4,0,3,1,1,1,2,2
714,2,1,6,1,1,1,2,2
715,3,1,2,1,1,3,3,2
...,...,...,...,...,...,...,...,...
3080170,1,1,3,2,4,1,2,6
3080388,11,0,4,1,4,3,5,3
3080389,14,0,1,3,4,2,2,6
3080390,20,0,2,1,4,2,2,2


In [ ]:
# Creating bins for "AGE" feature of the model

## labels = ['0-1', '1-10' '11-20']


bins = [-1, 1, 10, 21]  # Using -1 as the lower bound to include 0
labels = [1, 2, 3]

# Create a new column with the binned categories
df_kmode.loc[:, 'AGE'] = pd.cut(df_kmode['AGE'], bins=bins, labels=labels)

<ipython-input-63-67389005cdef>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kmode.loc[:, 'AGE'] = pd.cut(df_kmode['AGE'], bins=bins, labels=labels)
<ipython-input-63-67389005cdef>:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_kmode.loc[:, 'AGE'] = pd.cut(df_kmode['AGE'], bins=bins, labels=labels)


In [ ]:
#checking the results

df_kmode

,AGE,FEMALE,RACE,HOSP_REGION,ZIPINC_QRTL,LOS,TOTCHG,PAY1
711,1,0,6,2,1,1,1,2
712,2,0,3,1,1,2,3,2
713,2,0,3,1,1,1,2,2
714,2,1,6,1,1,1,2,2
715,2,1,2,1,1,3,3,2
...,...,...,...,...,...,...,...,...
3080170,1,1,3,2,4,1,2,6
3080388,3,0,4,1,4,3,5,3
3080389,3,0,1,3,4,2,2,6
3080390,3,0,2,1,4,2,2,2
